# Project: Wrangling and Analyze Data
___

# Data Gathering

In [88]:
import pandas as pd

In [89]:
import gather # gathering functions module, shift+tab on each function call bellow for more details.

1. ~Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)~

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

`gather.download_file_from_url(url_path='data/link.txt', destination='data/image_predictions.tsv')` **\*file already exists**

In [90]:
help(gather.download_file_from_url) # doc for the above function

Help on function download_file_from_url in module gather:

download_file_from_url(url_path, destination)
    Save File From a given url.
    
    :param url_path: (str) a path a file containing the url.
    :param destination: (str) a path to save the response at.



3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

**Auth object:**

In [91]:
from os.path import exists

if exists('data/auth.txt'):
    auth = gather.get_auth(auth_file='data/auth.txt') # get authentication
else:
    raise FileNotFoundError('Please fill your API credentials (without any spaces) at the empty auth file at data/auth_blank.txt')

In [92]:
# auth = gather.get_auth(auth_file='data/auth_blank.txt') # RUN ONLY AFTER FILLING CREDENTIALS IN FILE

**Archive data DataFrame:**

In [93]:
archive_original = pd.read_csv('data/twitter-archive-enhanced.csv', 
                               dtype={'tweet_id': pd.StringDtype()})
dump_lst = archive_original['tweet_id'] # to get list of id's as strings
archive_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   string 
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

***notice-loaded to json instead, relevant attributes only (rt and favorite counts):***

`gather.dump_tweets_data(id_lst=dump_lst, auth=auth)` **\*takes about 30 min, file already exists**

In [94]:
help(gather.dump_tweets_data) # doc for the above function

Help on function dump_tweets_data in module gather:

dump_tweets_data(id_lst, auth)
    Dump tweets' id, retweet count and favorite count into json file.
    
    :param id_lst: (list[str]) a list containing tweet id's.
    :param auth: (tweepy.auth.OAuth1UserHandler) an auth object.



**Tweets' counts DataFrame:**

In [95]:
tweet_cnts = pd.read_json('data/tweets_data.json', dtype={'id': pd.StringDtype()})
tweet_cnts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2356 non-null   string 
 1   favorite_count  2326 non-null   float64
 2   retweet_count   2326 non-null   float64
dtypes: float64(2), string(1)
memory usage: 55.3 KB


**image predictions DataFrame:** 

In [96]:
img_preds = pd.read_csv('data/image_predictions.tsv', sep='\t', 
                        dtype={'tweet_id': pd.StringDtype()})
img_preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   string 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(4), string(1)
memory usage: 152.1+ KB


---

# Assessing Data
(Each dataframe has been assessed seperately below)

# **`archive_original` issues:**
### Quality issues
- *completeness*
    - missing tweet links urls in `expanded_urls` data, even in non empty cells. Detected visually - `archive_original.iloc[313]`, `archive_original.iloc[35]`
    - because we would like to clearly see if the relevant id is an original tweet, a boolean column
    `is_original` should be included.
- *validity* - 
    - inappropriate data type for columns `in_reply_to_status_id`,                   
    `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`: float -> string
    - inappropriate data type for columns `timestamp`, `retweeted_status_timestamp`: object -> datetime. both issues detected by using programmatic detection- `archive_original.info()`
    - invalid denominators in `rating_denominators`: all should be 10
    - duplicated links within the same cell in `expanded_urls`. Detected visually - `archive_original['expanded_urls'].iloc[98]`
- *consistency* - 
    - name 'a' appears 55 times in the `name` column, probably representing unknown names, while there are defined None's in the column. Used visual, then programmatic detection-`archive_original['name'].value_counts()`
    
### Tidiness issues
- the 4 leftmost columns describe the same data, should be melted into one categorical column.
- `in_reply_to_status_id`, `in_reply_to_user_id` would be better reindexed to the left, as they consist mostly of NaN's and not really significant to the overall dataset
- retweeted data columns would be better seperated as a standalone dataframe.

In [97]:
archive_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   string 
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [98]:
archive_original.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


# **`img_preds` issues:**
Used both programmatical and visual assessments, did not observe any issues.
Also, no multiple records of same id's. One picture for each id.
Used programmatic detection: `len(img_copy[img_copy['tweet_id'].duplicated()])`

In [99]:
img_preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   string 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(4), string(1)
memory usage: 152.1+ KB


In [100]:
img_preds.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


# **`tweet_cnts` issues:**

### Tidiness issues
- dataframe would be better joined to archive_original, as it consists of general tweets data.

### Quality issues
- *validity*
    - count columns should be converted to int to avoid confusion.

In [101]:
tweet_cnts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2356 non-null   string 
 1   favorite_count  2326 non-null   float64
 2   retweet_count   2326 non-null   float64
dtypes: float64(2), string(1)
memory usage: 55.3 KB


In [102]:
tweet_cnts.head()

,id,favorite_count,retweet_count
0,892420643555336193,32991.0,6913.0
1,892177421306343426,28530.0,5208.0
2,891815181378084864,21448.0,3435.0
3,891689557279858688,35998.0,7122.0
4,891327558926688256,34424.0,7644.0


___
# Cleaning Data

In [103]:
# Make copies of original pieces of data
archive_copy = archive_original.copy()
img_copy = img_preds.copy()
tweet_copy = tweet_cnts.copy()

## `archive_original` cleaning:

### Issue #1: missing `expanded_urls` data.

#### Define: 
- concat `tweet_id` to WeRateDogs url prefix where `expanded_urls` is null or tweet url is missing:
    - first fill nan
    - then check entire column for missing pattern
    - finally, append a comma then the pattern to all missing it.

#### Code

In [104]:
val =  'https://twitter.com/dog_rates/status/' + archive_copy['tweet_id']
archive_copy['expanded_urls'] = archive_copy['expanded_urls'].fillna(value=val)

In [105]:
val = 'https://twitter.com/dog_rates/status/' + archive_copy['tweet_id']
archive_copy['expanded_urls'] = archive_copy['expanded_urls'].fillna(value=val)

In [106]:
mask = ~archive_copy['expanded_urls'].str.contains(r'https://twitter.com/dog_rates/status/\d+')
missing_locs = archive_copy[mask].index # useful for later testing
archive_copy.loc[missing_locs, 'expanded_urls'] += ',' + val

#### Test

In [107]:
all(archive_copy['expanded_urls'].str.contains(r'https://twitter.com/dog_rates/status/\d+'))

True

In [108]:
archive_copy.loc[missing_locs, 'expanded_urls'].iloc[0]

'https://twitter.com/4bonds2carbon/status/885517367337512960,https://twitter.com/dog_rates/status/885518971528720385'

### Issue #2: because we would like to clearly see if the relevant id is an original tweet, a boolean column is_original should be included.

#### Define
- if `retweeted_status_id` is null, then `is_original` is True, else False.

#### Code

In [109]:
archive_copy['is_original'] = archive_copy['retweeted_status_id'].isna()

#### Test

In [110]:
archive_copy[['retweeted_status_id', 'is_original']][~archive_copy['retweeted_status_id'].isna()]

,retweeted_status_id,is_original
19,8.874740e+17,False
32,8.860537e+17,False
36,8.305833e+17,False
68,8.780576e+17,False
73,8.782815e+17,False
...,...,...
1023,6.678667e+17,False
1043,6.671383e+17,False
1242,7.119983e+17,False
2259,6.675487e+17,False


### Issue #3: inappropriate data type for columns in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id: float -> string

#### Define
- iterate over the columns
- then, convert the dtype of each one to int in order to disable the scientific notation.
- finally, convert the dtype of each one to string.

#### Code

In [111]:
to_convert = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id']
for col in to_convert:
    archive_copy[col] = archive_copy[col].astype(pd.Int64Dtype()).astype(pd.StringDtype())

#### Test

In [112]:
archive_copy['in_reply_to_status_id'][archive_copy['in_reply_to_status_id'].notna()]

30      886266357075128320
55      881607037314052096
64      879553827334172672
113     870726202742493184
148     863425645568774144
               ...        
2038    671544874165002240
2149    669354382627049472
2169    667806454573760512
2189    668920717132582912
2298    667065535570550784
Name: in_reply_to_status_id, Length: 78, dtype: string

### Issue #4: inappropriate data type for columns `timestamp`, `retweeted_status_timestamp`: object -> datetime.

#### Define

- convert given columns to datetime using Pandas' `to_datetime` method.

#### Code

In [113]:
to_convert = ['timestamp', 'retweeted_status_timestamp']
for col in to_convert:
    archive_copy[col] = pd.to_datetime(archive_copy[col])

#### Test

In [114]:
archive_copy[to_convert].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   timestamp                   2356 non-null   datetime64[ns, UTC]
 1   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2)
memory usage: 36.9 KB


In [115]:
archive_copy[to_convert]

,timestamp,retweeted_status_timestamp
0,2017-08-01 16:23:56+00:00,NaT
1,2017-08-01 00:17:27+00:00,NaT
2,2017-07-31 00:18:03+00:00,NaT
3,2017-07-30 15:58:51+00:00,NaT
4,2017-07-29 16:00:24+00:00,NaT
...,...,...
2351,2015-11-16 00:24:50+00:00,NaT
2352,2015-11-16 00:04:52+00:00,NaT
2353,2015-11-15 23:21:54+00:00,NaT
2354,2015-11-15 23:05:30+00:00,NaT


### Issue #5: invalid denominators in `rating_denominators`: all should be 10.

#### Define

- Coerce the integer value `10` on the entire column, using Pandas' `map` function.

#### Code

In [116]:
archive_copy['rating_denominator'] = archive_copy['rating_denominator'].map(lambda item: 10)

#### Test

In [117]:
archive_copy['rating_denominator'].value_counts(dropna=False)

10    2356
Name: rating_denominator, dtype: int64

### Issue #6: duplicated links within the same cell in `expanded_url`.

#### Define

- split strings at `,` to list.
- set() on the output.
- join the set using `,`
- apply to column using Pandas' `map` function.

#### Code

In [118]:
archive_copy['expanded_urls'].iloc[98] # useful for later testing

'https://www.gofundme.com/help-my-baby-sierra-get-better,https://twitter.com/dog_rates/status/873213775632977920/photo/1,https://twitter.com/dog_rates/status/873213775632977920/photo/1'

In [119]:
def unique_url(urls: str):
    return ','.join(set(urls.split(',')))

In [120]:
archive_copy['expanded_urls'] = archive_copy['expanded_urls'].map(unique_url)

#### Test

In [121]:
archive_copy['expanded_urls'].iloc[98]

'https://www.gofundme.com/help-my-baby-sierra-get-better,https://twitter.com/dog_rates/status/873213775632977920/photo/1'

### Issue #7: name 'a' appears 55 times in the `name` column, probably representing unknown names, while there are defined None's in the column.

#### Define
- check if cell value is 'a' or 'None'
- if true, convert to Pandas' `NA`
- else, do nothing.
- apply to column using Pandas' `map`.

In [122]:
def nullify(cell: str):
    if cell in ('a', 'None'):
        return pd.NA
    return cell

#### Code

In [123]:
archive_copy['name'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2356 entries, 0 to 2355
Series name: name
Non-Null Count  Dtype 
--------------  ----- 
2356 non-null   object
dtypes: object(1)
memory usage: 18.5+ KB


In [124]:
def nullify(cell: str):
    if cell in ('a', 'None'):
        return pd.NA
    return cell

In [125]:
archive_copy['name'] = archive_copy['name'].map(nullify)

#### Test

In [126]:
archive_copy['name'].value_counts(dropna=False)

<NA>          800
Charlie        12
Cooper         11
Lucy           11
Oliver         11
             ... 
Aqua            1
Chase           1
Meatball        1
Rorie           1
Christoper      1
Name: name, Length: 956, dtype: int64

### Issue #8: the 4 leftmost columns describe the same data, should be melted into one categorical column.

#### Define
- Use Pandas' `melt` function to melt the columns into a category and value column.
- create in a temporary dataframe that consists of only the records which do have a category(not 'None').
- drop its excess `value` column
- instead of simply removing duplicated id's, format the categories column in the temporary dataframe to include multiple categories for each id, as each tweet can validly contain multiple categories.
- convert temp to dtype string for proper nullable data type on left join.
- ***NOTE - AS records may have some combination of categories, a string dtype instead would be more approppriate.***
- left join `archive_copy` with temp on `tweet_id`
- drop the old category columns from `archive_copy`


#### Code

In [127]:
temp = pd.melt(archive_copy, id_vars='tweet_id', 
               value_vars=['doggo', 'floofer', 'pupper', 'puppo'], var_name='categories')
temp = temp[temp['value'] != 'None'].drop(labels='value', axis=1)

In [128]:
temp[temp['tweet_id'] == '817777686764523521'] # a duplicated id example

,tweet_id,categories
460,817777686764523521,doggo
5172,817777686764523521,pupper


In [129]:
originals = temp[~temp['tweet_id'].duplicated()]['tweet_id'] # original id only
duplicates = temp[temp['tweet_id'].duplicated()]['tweet_id'] # duplicated id only
for i in originals.index:
    duplicates_of_item_locs = duplicates[duplicates == originals.loc[i]].index
    items_to_add = temp.loc[duplicates_of_item_locs, 'categories']
    if items_to_add.size > 0:
        temp.loc[i, 'categories'] += ',' + ','.join(items_to_add)
temp = temp.drop_duplicates(subset='tweet_id').astype(pd.StringDtype())

In [130]:
drop_lst = ['doggo', 'floofer', 'pupper', 'puppo']
archive_copy = archive_copy.merge(right=temp, on='tweet_id', how='left').drop(labels=drop_lst, axis=1)

#### Test

In [131]:
archive_copy['categories'].value_counts(dropna=False)

<NA>             1976
pupper            245
doggo              83
puppo              29
doggo,pupper       12
floofer             9
doggo,puppo         1
doggo,floofer       1
Name: categories, dtype: Int64

In [132]:
archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   string             
 1   in_reply_to_status_id       78 non-null     string             
 2   in_reply_to_user_id         78 non-null     string             
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    string             
 7   retweeted_status_user_id    181 non-null    string             
 8   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 9   expanded_urls               2356 non-null   object             
 10  rating_numerator            2356 non-null   int64           

### Issue #9: `in_reply_to_status_id`, `in_reply_to_user_id` would be better reindexed to the left, as they consist mostly of NaN's and not really significant to the overall dataset (NOTE - ISSUE REOPENED BELOW)

#### Define:

- **NOTE:** This issue should be taken care of later, as more dataframes may be joined.
- Then, prioritize the entire set of columns from left to right.

### Issue #10: retweeted data columns would be better separated as a standalone dataframe.

#### Define:

- create a DataFrame from these columns
- include only retweets(`is_original` False)
- drop columns from `archive_copy`.

#### Code:

## NEW `retweet_data` DataFrame:

In [133]:
col_lst = ['tweet_id', 'retweeted_status_id', 
           'retweeted_status_user_id', 'retweeted_status_timestamp']
mask = ~archive_copy['is_original']
retweet_data = archive_copy[col_lst][mask].copy() # new dataframe

In [134]:
archive_copy = archive_copy.drop(axis=1, labels=col_lst[1:]) # not dropping id column

#### Test:

In [135]:
retweet_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 19 to 2260
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    181 non-null    string             
 1   retweeted_status_id         181 non-null    string             
 2   retweeted_status_user_id    181 non-null    string             
 3   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), string(3)
memory usage: 7.1 KB


In [136]:
archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               2356 non-null   string             
 1   in_reply_to_status_id  78 non-null     string             
 2   in_reply_to_user_id    78 non-null     string             
 3   timestamp              2356 non-null   datetime64[ns, UTC]
 4   source                 2356 non-null   object             
 5   text                   2356 non-null   object             
 6   expanded_urls          2356 non-null   object             
 7   rating_numerator       2356 non-null   int64              
 8   rating_denominator     2356 non-null   int64              
 9   name                   1556 non-null   object             
 10  is_original            2356 non-null   bool               
 11  categories             380 non-null    string           

## `tweet_cnts` cleaning:

### Issue #11: `tweet_cnts` would be better joined to archive_original, as it consists of general tweets data.

#### Define:

- left join `archive_copy` and `tweet_copy` on id.
- drop excess id column.

#### Code:

In [137]:
tweet_copy.info() # check name of id column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2356 non-null   string 
 1   favorite_count  2326 non-null   float64
 2   retweet_count   2326 non-null   float64
dtypes: float64(2), string(1)
memory usage: 55.3 KB


In [138]:
temp = archive_copy.merge(right=tweet_copy, how='left', left_on='tweet_id', right_on='id')
archive_copy = temp.drop(axis=1, labels='id')

#### Test:

In [139]:
archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               2356 non-null   string             
 1   in_reply_to_status_id  78 non-null     string             
 2   in_reply_to_user_id    78 non-null     string             
 3   timestamp              2356 non-null   datetime64[ns, UTC]
 4   source                 2356 non-null   object             
 5   text                   2356 non-null   object             
 6   expanded_urls          2356 non-null   object             
 7   rating_numerator       2356 non-null   int64              
 8   rating_denominator     2356 non-null   int64              
 9   name                   1556 non-null   object             
 10  is_original            2356 non-null   bool               
 11  categories             380 non-null    string           

### Issue #12: count columns should be converted to int to avoid confusion.

#### Define:

- Address the issue in `archive_copy', as it includes the tweet counts dataframe.
- convert columns to Pandas' `Int64Dtype`

#### Code:

In [140]:
convert_map = {'favorite_count':pd.Int64Dtype(), 'retweet_count': pd.Int64Dtype()}
archive_copy = archive_copy.astype(dtype=convert_map)

#### Test:

In [141]:
archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               2356 non-null   string             
 1   in_reply_to_status_id  78 non-null     string             
 2   in_reply_to_user_id    78 non-null     string             
 3   timestamp              2356 non-null   datetime64[ns, UTC]
 4   source                 2356 non-null   object             
 5   text                   2356 non-null   object             
 6   expanded_urls          2356 non-null   object             
 7   rating_numerator       2356 non-null   int64              
 8   rating_denominator     2356 non-null   int64              
 9   name                   1556 non-null   object             
 10  is_original            2356 non-null   bool               
 11  categories             380 non-null    string           

## Bonus

## REVISITED - Issue #9 - reindex all columns by priority left to right.

#### Define:

- reorder columns using reindex method.

#### Code:

In [142]:
top = 'tweet_id', 'is_original', 'timestamp', 'favorite_count', 'retweet_count'
bottom = 'in_reply_to_status_id', 'in_reply_to_user_id'
col_lst = [item for item in archive_copy.columns if item not in bottom and item not in top]
ordered_labels = top + tuple(col_lst) + bottom
archive_copy = archive_copy.reindex(axis=1, labels=ordered_labels)

#### Test:

In [143]:
archive_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               2356 non-null   string             
 1   is_original            2356 non-null   bool               
 2   timestamp              2356 non-null   datetime64[ns, UTC]
 3   favorite_count         2326 non-null   Int64              
 4   retweet_count          2326 non-null   Int64              
 5   source                 2356 non-null   object             
 6   text                   2356 non-null   object             
 7   expanded_urls          2356 non-null   object             
 8   rating_numerator       2356 non-null   int64              
 9   rating_denominator     2356 non-null   int64              
 10  name                   1556 non-null   object             
 11  categories             380 non-null    string           

---
## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

### Joining into master dataset:

In [144]:
# a df without retweet data
master_df = archive_copy.merge(right=img_copy, on='tweet_id', how='left')

### Bonus Issue #13 - use accurate type for each column

#### Define:

- Although most column dtypes are accurate, should convert `object` -> `pd.StringDtype`.

In [145]:
for col in master_df:
    if master_df[col].dtype == 'object':
        master_df[col] = master_df[col].astype(pd.StringDtype())

In [146]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               2356 non-null   string             
 1   is_original            2356 non-null   bool               
 2   timestamp              2356 non-null   datetime64[ns, UTC]
 3   favorite_count         2326 non-null   Int64              
 4   retweet_count          2326 non-null   Int64              
 5   source                 2356 non-null   string             
 6   text                   2356 non-null   string             
 7   expanded_urls          2356 non-null   string             
 8   rating_numerator       2356 non-null   int64              
 9   rating_denominator     2356 non-null   int64              
 10  name                   1556 non-null   string             
 11  categories             380 non-null    string           

## Finally, storing into:

In [60]:
master_df.to_csv("cleaned_data/twitter_archive_master.csv", index=False)
retweet_data.to_csv("cleaned_data/retweet_data.csv", index=False)

---
## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization